<a href="https://colab.research.google.com/github/AIM-Harvard/aimi_alpha/blob/main/aimi/nnunet_prostate/notebooks/nnunet_prostate_mwe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ModelHub - Prostate Segmentation**

This notebook provides an example of how to run an end-to-end (cloud-based) data analysis using the nnU-Net prostate segmentation model.

The way all the operations are executed - from pulling data to data postprocessing and the standardisation of the results - have the goal of promoting transparency and reproducibility.

## **Environment Setup**

This demo notebook is intended to be run using a GPU.

To access a free GPU on Colab:
`Edit > Notebooks Settings`.

From the dropdown menu under `Hardware accelerator`, select `GPU`. Let's check the Colab instance is indeed equipped with a GPU.

In [1]:
import os
import sys
import shutil

import yaml

import time
import tqdm


# useful information
curr_dir = !pwd
curr_droid = !hostname
curr_pilot = !whoami

print(time.asctime(time.localtime()))

print("\nCurrent directory :", curr_dir[-1])
print("Hostname          :", curr_droid[-1])
print("Username          :", curr_pilot[-1])

print("Python version    :", sys.version.split('\n')[0])

Tue Oct 11 12:49:19 2022

Current directory : /content
Hostname          : 4e3b3a57ea7c
Username          : root
Python version    : 3.7.14 (default, Sep  8 2022, 00:06:44) 


The authentication to Google is necessary to run BigQuery queries.

Every operation throughout the whole notebook (BigQuery, fetching data from the IDC buckets) is completely free. The only thing that is needed in order to run the notebook is the set-up of a Google Cloud project. In order for the notebook to work as intended, you will need to specify the name of the project in the cell after the authentication one.

In [2]:
from google.colab import auth
auth.authenticate_user()

In [3]:
from google.colab import files
from google.cloud import storage
from google.cloud import bigquery as bq

# INSERT THE ID OF YOUR PROJECT HERE!
project_id = "idc-sandbox-000"

Throughout this Colab notebook, for image pre-processing we will use [Plastimatch](https://plastimatch.org), a reliable and open source software for image computation. We will be running Plastimatch using the simple [PyPlastimatch](https://github.com/AIM-Harvard/pyplastimatch/tree/main/pyplastimatch) python wrapper. 

In [4]:
%%capture
!apt install plastimatch

In [5]:
# check plastimatch was correctly installed
!plastimatch --version

plastimatch version 1.7.0


---

Start by cloning the AIMI hub repository on the Colab instance.

The AIMI hub repository stores all the code we will use for pulling, preprocessing, processing, and postprocessing the data for this use case - as long as the others shared through AIMI hub.

In [6]:
!git clone https://github.com/AIM-Harvard/aimi_alpha.git aimi

Cloning into 'aimi'...
remote: Enumerating objects: 439, done.
remote: Counting objects: 100% (77/77), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 439 (delta 33), reused 40 (delta 14), pack-reused 362
Receiving objects: 100% (439/439), 3.71 MiB | 10.51 MiB/s, done.
Resolving deltas: 100% (227/227), done.


To organise the DICOM data in a more common (and human-understandable) fashion after downloading those from the buckets, we will make use of [DICOMSort](https://github.com/pieper/dicomsort). 

DICOMSort is an open source tool for custom sorting and renaming of dicom files based on their specific DICOM tags. In our case, we will exploit DICOMSort to organise the DICOM data by `PatientID` and `Modality` - so that the final directory will look like the following:

```
data/raw/nsclc-radiomics/dicom/$PatientID
 └─── CT
       ├─── $SOPInstanceUID_slice0.dcm
       ├─── $SOPInstanceUID_slice1.dcm
       ├───  ...
       │
      RTSTRUCT 
       ├─── $SOPInstanceUID_RTSTRUCT.dcm
      SEG
       └─── $SOPInstanceUID_RTSEG.dcm

```

In [7]:
!git clone https://github.com/pieper/dicomsort dicomsort

Cloning into 'dicomsort'...
remote: Enumerating objects: 130, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 130 (delta 0), reused 1 (delta 0), pack-reused 126
Receiving objects: 100% (130/130), 44.12 KiB | 426.00 KiB/s, done.
Resolving deltas: 100% (63/63), done.


We will also use DCMQI for converting the resulting segmentation into standard DICOM SEG objects.

In [8]:
%%capture
dcmqi_release_url = "https://github.com/QIICR/dcmqi/releases/download/v1.2.4/dcmqi-1.2.4-linux.tar.gz"
dcmqi_download_path = "/content/dcmqi-1.2.4-linux.tar.gz"
dcmqi_path = "/content/dcmqi-1.2.4-linux"

!wget -O $dcmqi_download_path $dcmqi_release_url

!tar -xvf $dcmqi_download_path

!mv $dcmqi_path/bin/* /bin

---

In [9]:
%%capture
!pip install pyplastimatch nnunet ipywidgets

In [63]:
import shutil
import random

import json
import pprint
import numpy as np
import pandas as pd

import pydicom
import nibabel as nib
import SimpleITK as sitk
import pyplastimatch as pypla

print("Python version               : ", sys.version.split('\n')[0])
print("Numpy version                : ", np.__version__)

# ----------------------------------------

#everything that has to do with plotting goes here below
import matplotlib
matplotlib.use("agg")

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib.patches import Patch

%matplotlib inline
%config InlineBackend.figure_format = "png"

import ipywidgets as ipyw

## ----------------------------------------

# create new colormap appending the alpha channel to the selected one
# (so that we don't get a \"color overlay\" when plotting the segmask superimposed to the MR)
cmap = plt.cm.Reds
my_reds = cmap(np.arange(cmap.N))
my_reds[:, -1] = np.linspace(0, 1, cmap.N)
my_reds = ListedColormap(my_reds)

cmap = plt.cm.Greens
my_greens = cmap(np.arange(cmap.N))
my_greens[:, -1] = np.linspace(0, 1, cmap.N)
my_greens = ListedColormap(my_greens)

cmap = plt.cm.Blues
my_blues = cmap(np.arange(cmap.N))
my_blues[:, -1] = np.linspace(0, 1, cmap.N)
my_blues = ListedColormap(my_blues)

cmap = plt.cm.spring
my_spring = cmap(np.arange(cmap.N))
my_spring[:, -1] = np.linspace(0, 1, cmap.N)
my_spring = ListedColormap(my_spring)
## ----------------------------------------

import seaborn as sns

Python version               :  3.7.14 (default, Sep  8 2022, 00:06:44) 
Numpy version                :  1.21.6


Provided everything was set up correctly, we can run the BigQuery query and get all the information we need to download the testing data from the IDC platform.

For this specific use case, we are going to be working with the Prostate-3T collection (T2w Prostate MR scans of prostate cancer patients).

In [11]:
%%bigquery --project=$project_id cohort_df

SELECT
  dicom_pivot_v11.PatientID,
  dicom_pivot_v11.collection_id,
  dicom_pivot_v11.source_DOI,
  dicom_pivot_v11.StudyInstanceUID,
  dicom_pivot_v11.SeriesInstanceUID,
  dicom_pivot_v11.SOPInstanceUID,
  dicom_pivot_v11.gcs_url
FROM
  `bigquery-public-data.idc_v11.dicom_pivot_v11` dicom_pivot_v11
WHERE
  StudyInstanceUID IN (
    SELECT
      StudyInstanceUID
    FROM
      `bigquery-public-data.idc_v11.dicom_pivot_v11` dicom_pivot_v11
    WHERE
      (
        dicom_pivot_v11.collection_id IN ('Community', 'prostate_3t')
      )
    GROUP BY
      StudyInstanceUID
  )
GROUP BY
  dicom_pivot_v11.PatientID,
  dicom_pivot_v11.collection_id,
  dicom_pivot_v11.source_DOI,
  dicom_pivot_v11.StudyInstanceUID,
  dicom_pivot_v11.SeriesInstanceUID,
  dicom_pivot_v11.SOPInstanceUID,
  dicom_pivot_v11.gcs_url
ORDER BY
  dicom_pivot_v11.PatientID ASC,
  dicom_pivot_v11.collection_id ASC,
  dicom_pivot_v11.source_DOI ASC,
  dicom_pivot_v11.StudyInstanceUID ASC,
  dicom_pivot_v11.SeriesInstanceUID ASC,
  dicom_pivot_v11.SOPInstanceUID ASC,
  dicom_pivot_v11.gcs_url ASC

In [12]:
# this works as intended only if the BQ query parses data from a single dataset
# if not, feel free to set the name manually!
dataset_name = cohort_df["collection_id"].values[0]

In [13]:
# create the directory tree
!mkdir -p data models

!mkdir -p data/raw 
!mkdir -p data/raw/tmp data/raw/$dataset_name
!mkdir -p data/raw/$dataset_name/dicom

!mkdir -p data/processed
!mkdir -p data/processed/$dataset_name
!mkdir -p data/processed/$dataset_name/nii
!mkdir -p data/processed/$dataset_name/dicomseg

!mkdir -p data/model_input/
!mkdir -p data/nnunet_output/

Download the segmentation model(s) from Zenodo (the original source chosen from the nnU-Net developers).

Note that this can either be very fast (2 minutes or even less, at speeds around 40MB/s) or very slow (up to 10 minutes, or sometimes more) - probably depending on the traffic on the Zenodo's end and other factors. If the download is taking a long time after the first minute or so, consider interrupting the cell execution and running the cell again.

In [14]:
seg_model_url = "https://zenodo.org/record/4003545/files/Task024_Promise.zip?download=1"
model_download_path = "/content/models/Task024_Promise.zip"

!wget -O $model_download_path $seg_model_url

--2022-10-11 12:50:40--  https://zenodo.org/record/4003545/files/Task024_Promise.zip?download=1
Resolving zenodo.org (zenodo.org)... 188.184.117.155
Connecting to zenodo.org (zenodo.org)|188.184.117.155|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2258526938 (2.1G) [application/octet-stream]
Saving to: ‘/content/models/Task024_Promise.zip’

/content/models/Tas 100%[===================>]   2.10G  4.89MB/s    in 4m 42s  

2022-10-11 12:55:25 (7.63 MB/s) - ‘/content/models/Task024_Promise.zip’ saved [2258526938/2258526938]



Initialize the environment variables nnU-Net needs.

In [15]:
os.environ["RESULTS_FOLDER"] = "/content/data/nnunet_output/"
os.environ["WEIGHTS_FOLDER"] = "/content/data/nnunet_output/nnUNet"

In [16]:
%%capture
!nnUNet_install_pretrained_model_from_zip $model_download_path

## **Parsing Cohort Information from BigQuery Tables**

We can check the various fields of the table we populated by running the BigQuery query.

This table will store one entry for each DICOM file in the dataset (therefore, expect thousands of rows!)

In [17]:
pat_id_list = sorted(list(set(cohort_df["PatientID"].values)))

print("Total number of unique Patient IDs:", len(pat_id_list))

display(cohort_df.info())

display(cohort_df.head())

Total number of unique Patient IDs: 64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1258 entries, 0 to 1257
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   PatientID          1258 non-null   object
 1   collection_id      1258 non-null   object
 2   source_DOI         1258 non-null   object
 3   StudyInstanceUID   1258 non-null   object
 4   SeriesInstanceUID  1258 non-null   object
 5   SOPInstanceUID     1258 non-null   object
 6   gcs_url            1258 non-null   object
dtypes: object(7)
memory usage: 68.9+ KB


None

,PatientID,collection_id,source_DOI,StudyInstanceUID,SeriesInstanceUID,SOPInstanceUID,gcs_url
0,Prostate3T-01-0001,prostate_3t,10.7937/K9/TCIA.2015.QJTV5IL5,1.3.6.1.4.1.14519.5.2.1.7308.2101.878027069026...,1.3.6.1.4.1.14519.5.2.1.7308.2101.294123309272...,1.3.6.1.4.1.14519.5.2.1.7308.2101.111446010896...,gs://public-datasets-idc/df6e142e-5e32-475f-bb...
1,Prostate3T-01-0001,prostate_3t,10.7937/K9/TCIA.2015.QJTV5IL5,1.3.6.1.4.1.14519.5.2.1.7308.2101.878027069026...,1.3.6.1.4.1.14519.5.2.1.7308.2101.294123309272...,1.3.6.1.4.1.14519.5.2.1.7308.2101.126784618763...,gs://public-datasets-idc/e7f43155-639c-4fa6-bc...
2,Prostate3T-01-0001,prostate_3t,10.7937/K9/TCIA.2015.QJTV5IL5,1.3.6.1.4.1.14519.5.2.1.7308.2101.878027069026...,1.3.6.1.4.1.14519.5.2.1.7308.2101.294123309272...,1.3.6.1.4.1.14519.5.2.1.7308.2101.175171821828...,gs://public-datasets-idc/52b31bf9-b878-4abf-97...
3,Prostate3T-01-0001,prostate_3t,10.7937/K9/TCIA.2015.QJTV5IL5,1.3.6.1.4.1.14519.5.2.1.7308.2101.878027069026...,1.3.6.1.4.1.14519.5.2.1.7308.2101.294123309272...,1.3.6.1.4.1.14519.5.2.1.7308.2101.182499317437...,gs://public-datasets-idc/25129d39-9081-417e-9a...
4,Prostate3T-01-0001,prostate_3t,10.7937/K9/TCIA.2015.QJTV5IL5,1.3.6.1.4.1.14519.5.2.1.7308.2101.878027069026...,1.3.6.1.4.1.14519.5.2.1.7308.2101.294123309272...,1.3.6.1.4.1.14519.5.2.1.7308.2101.215493747024...,gs://public-datasets-idc/deeef10f-0638-4beb-b7...


---

## **Set Run Parameters**

From this cell, we can configure the nnU-Net inference step - specifying, for instance, the type of model we want to run (among the four different models the framework provides), whether we want to use test time augmentation, or whether we want to export the soft probability maps of the segmentation masks.


In [50]:
# FIXED PARAMETERS
data_base_path = "/content/data"
raw_base_path = "/content/data/raw/tmp"
sorted_base_path = os.path.join("/content/data/raw/", dataset_name, "dicom")

processed_base_path = os.path.join("/content/data/processed/", dataset_name)
processed_nifti_path = os.path.join(processed_base_path, "nii")

processed_dicomseg_path = os.path.join(processed_base_path, "dicomseg")

model_input_folder = "/content/data/model_input/"
model_output_folder = "/content/data/nnunet_output/"

dicomseg_json_path = "/content/aimi/aimi/nnunet_prostate/config/dicomseg_metadata.json"

# -----------------
# nnU-Net pipeline parameters

# choose from: "2d", "3d_lowres", "3d_fullres", "3d_cascade_fullres"
nnunet_model = "2d"
use_tta = True

## **Running the Analysis for a Single Patient**

In [44]:
import aimi.aimi as aimi

from aimi import general_utils as aimi_utils
from aimi import nnunet_prostate as aimi_model

The following cells run all the processing pipeline, from pre-processing to post-processing.

In [28]:
pat_id = random.choice(cohort_df["PatientID"].values)
pat_df = cohort_df[cohort_df["PatientID"] == pat_id].reset_index(drop = True)

display(pat_df.info())
display(pat_df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   PatientID          19 non-null     object
 1   collection_id      19 non-null     object
 2   source_DOI         19 non-null     object
 3   StudyInstanceUID   19 non-null     object
 4   SeriesInstanceUID  19 non-null     object
 5   SOPInstanceUID     19 non-null     object
 6   gcs_url            19 non-null     object
dtypes: object(7)
memory usage: 1.2+ KB


None

,PatientID,collection_id,source_DOI,StudyInstanceUID,SeriesInstanceUID,SOPInstanceUID,gcs_url
0,Prostate3T-01-0025,prostate_3t,10.7937/K9/TCIA.2015.QJTV5IL5,1.3.6.1.4.1.14519.5.2.1.7308.2101.880756110289...,1.3.6.1.4.1.14519.5.2.1.7308.2101.204499664621...,1.3.6.1.4.1.14519.5.2.1.7308.2101.102668792323...,gs://public-datasets-idc/26bd0c08-8f1a-4431-bf...
1,Prostate3T-01-0025,prostate_3t,10.7937/K9/TCIA.2015.QJTV5IL5,1.3.6.1.4.1.14519.5.2.1.7308.2101.880756110289...,1.3.6.1.4.1.14519.5.2.1.7308.2101.204499664621...,1.3.6.1.4.1.14519.5.2.1.7308.2101.105182557188...,gs://public-datasets-idc/4bb07403-0af7-4dbf-aa...
2,Prostate3T-01-0025,prostate_3t,10.7937/K9/TCIA.2015.QJTV5IL5,1.3.6.1.4.1.14519.5.2.1.7308.2101.880756110289...,1.3.6.1.4.1.14519.5.2.1.7308.2101.204499664621...,1.3.6.1.4.1.14519.5.2.1.7308.2101.131762566306...,gs://public-datasets-idc/0b796a6c-304d-4bc4-bc...
3,Prostate3T-01-0025,prostate_3t,10.7937/K9/TCIA.2015.QJTV5IL5,1.3.6.1.4.1.14519.5.2.1.7308.2101.880756110289...,1.3.6.1.4.1.14519.5.2.1.7308.2101.204499664621...,1.3.6.1.4.1.14519.5.2.1.7308.2101.140819923234...,gs://public-datasets-idc/b5909cb7-b598-4e5f-89...
4,Prostate3T-01-0025,prostate_3t,10.7937/K9/TCIA.2015.QJTV5IL5,1.3.6.1.4.1.14519.5.2.1.7308.2101.880756110289...,1.3.6.1.4.1.14519.5.2.1.7308.2101.204499664621...,1.3.6.1.4.1.14519.5.2.1.7308.2101.147873275407...,gs://public-datasets-idc/c6265a96-5235-4885-a4...


In [29]:
# init

print("Processing patient: %s"%(pat_id))

patient_df = cohort_df[cohort_df["PatientID"] == pat_id]

dicomseg_fn = pat_id + "_SEG.dcm"

input_nifti_fn = pat_id + "_0000.nii.gz"
input_nifti_path = os.path.join(model_input_folder, input_nifti_fn)

pred_nifti_fn = pat_id + ".nii.gz"
pred_nifti_path = os.path.join(model_output_folder, pred_nifti_fn)

pred_softmax_folder_name = "pred_softmax"
pred_softmax_folder_path = os.path.join(processed_nifti_path, pat_id, pred_softmax_folder_name)

Processing patient: Prostate3T-01-0025


In [30]:
# data cross-loading
aimi_utils.gcs.download_patient_data(raw_base_path = raw_base_path,
                                     sorted_base_path = sorted_base_path,
                                     patient_df = patient_df,
                                     remove_raw = True)

Copying files from IDC buckets to /content/data/raw/tmp/Prostate3T-01-0025...
Done in 15.4719 seconds.

Sorting DICOM files...
Done in 0.372711 seconds.
Sorted DICOM data saved at: /content/data/raw/prostate_3t/dicom/Prostate3T-01-0025
Removing un-sorted data at /content/data/raw/tmp/Prostate3T-01-0025...
... Done.


In [41]:
# DICOM MR to NIfTI - required for the processing
aimi_utils.preprocessing.pypla_dicom_mr_to_nifti(sorted_base_path = sorted_base_path,
                                                 processed_nifti_path = processed_nifti_path,
                                                 pat_id = pat_id, verbose = True)


Running 'plastimatch convert' with the specified arguments:
  --input /content/data/raw/prostate_3t/dicom/Prostate3T-01-0025/MR
  --output-img /content/data/processed/prostate_3t/nii/Prostate3T-01-0025/Prostate3T-01-0025_MR.nii.gz
... Done.


In [45]:
# prepare the `model_input` folder for the inference phase
aimi_utils.preprocessing.prep_mr_input_data(processed_nifti_path = processed_nifti_path,
                                            model_input_folder = model_input_folder,
                                            pat_id = pat_id)

Copying /content/data/processed/prostate_3t/nii/Prostate3T-01-0025/Prostate3T-01-0025_MR.nii.gz
to /content/data/model_input/Prostate3T-01-0025_0000.nii.gz...
... Done.


In [51]:
# run the DL-based prediction
aimi_model.utils.processing.process_patient_nnunet(model_input_folder = model_input_folder,
                                                   model_output_folder = model_output_folder, 
                                                   nnunet_model = nnunet_model, use_tta = use_tta)

Running `nnUNet_predict` with `2d` model...
Processing file at /content/data/model_input/Prostate3T-01-0025_0000.nii.gz...
Done in 21.5678 seconds.


In [52]:
model_output_seg = os.path.join(model_output_folder, pat_id + ".nii.gz")
seg_nii_path = os.path.join(processed_nifti_path, pat_id, pat_id + "_pred_prostate.nii.gz")

shutil.copyfile(model_output_seg, seg_nii_path)

'/content/data/processed/prostate_3t/nii/Prostate3T-01-0025/Prostate3T-01-0025_pred_prostate.nii.gz'

Before running the conversion to DICOM SEG, update the JSON file used to generate the DICOM SEG object with the configuration of nnU-Net we are going to be using for the specific run.

In [53]:
extended_model_name = "nnU-Net " + nnunet_model + "_tta" if use_tta else "nnU-Net " + nnunet_model

aimi_utils.config.modify_dicomseg_json_file(dicomseg_json_path = dicomseg_json_path,
                                            new_dicomseg_json_path = dicomseg_json_path,
                                            SegmentAlgorithmName = extended_model_name)

In [59]:
aimi_model.utils.postprocessing.nifti_to_dicomseg(sorted_base_path = sorted_base_path,
                                                  processed_base_path = processed_base_path,
                                                  dicomseg_json_path = dicomseg_json_path,
                                                  pat_id = pat_id)

---

In [60]:
from pyplastimatch.utils import viz as viz_utils

In [61]:
mr_nii_path = os.path.join("/content/data/processed/%s/nii/"%dataset_name, pat_id, pat_id + "_MR.nii.gz")
seg_nii_path = os.path.join("/content/data/nnunet_output/", pat_id + ".nii.gz")
softseg_nii_path = os.path.join("/content/data/nnunet_output/", pat_id + ".npz")

"""
alternative way of loading the resulting NIfTI files
nibabel can sometimes take better care of the orientation of the
converted/segmented images, but will orient the data differently by default
"""

#mr_nii = nib.load(mr_nii_path).dataobj
#seg_nii = nib.load(seg_nii_path).dataobj

mr_nii = sitk.GetArrayFromImage(sitk.ReadImage(mr_nii_path))
seg_nii = sitk.GetArrayFromImage(sitk.ReadImage(seg_nii_path))

In the next cell, we can visualise the result using a simple widget.

In [64]:
_ = viz_utils.AxialSliceSegmaskViz(ct_volume = mr_nii, 
                                   segmask_dict = {"prostate" : seg_nii},
                                   segmask_cmap_dict = {"prostate" : my_greens},
                                   dpi = 100, figsize = (8, 8),
                                   min_hu = -1024, max_hu = 1024)

interactive(children=(Output(),), _dom_classes=('widget-interact',))

---

## **Data Download**

In [65]:
%%capture

archive_fn = "%s.zip"%(pat_id)

try:
  os.remove(archive_fn)
except OSError:
  pass

seg_dicom_path = os.path.join(processed_dicomseg_path, pat_id, dicomseg_fn)
mr_dicom_path = os.path.join(sorted_base_path, pat_id)

!zip -j -r $archive_fn $mr_dicom_path $seg_dicom_path

In [66]:
filesize = os.stat(archive_fn).st_size/1024e03
print('Starting the download of "%s" (%2.1f MB)...\n'%(archive_fn, filesize))

files.download(archive_fn)

Starting the download of "Prostate3T-01-0025.zip" (2.8 MB)...



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>